In [1]:
import sys
sys.path.append('../preprocess/')
sys.path.append('../evaluate/')

import cleanup
import prepare
import evaluate
import baseline
from baseline import BaselineClassifier

import importlib
importlib.reload(cleanup)
importlib.reload(prepare)
importlib.reload(evaluate)
importlib.reload(baseline)


import os
import re

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.pipeline import Pipeline

X_all, y_all = cleanup.load_for_baseline()

price                           float64
starRating                      float64
category                         object
totalNrOfReviews                float64
installs                         object
contentRating                    object
appUrl                           object
libraries                       float64
bigCompany                      float64
daysSinceLastUpdated            float64
text                             object
1starReviews                    float64
2starReviews                    float64
3starReviews                    float64
4starReviews                    float64
5starReviews                    float64
requiredAndroidVersion_major    float64
requiredAndroidVersion_minor    float64
similarAppsAvgPrice             float64
dtype: object
(25927, 19) transformed data
Example rows with similarAppsAvgPrice equal nan
Series([], Name: appUrl, dtype: object)
Example rows with similarAppsAvgPrice equal NaN
Series([], Name: appUrl, dtype: object)
5796 samples dropped 

In [2]:
importlib.reload(cleanup)
importlib.reload(prepare)
importlib.reload(evaluate)
importlib.reload(baseline)
from baseline import BaselineClassifier


# use this as a prediction for the price of the original data
###################################################################

X_train, X_eval, y_train, y_eval = train_test_split(X_all, y_all)

# just to be save
X_train = X_train.copy()
X_eval = X_eval.copy()
y_train = y_train.copy()
y_eval = y_eval.copy()


q25, q50, q75 = prepare.price_quartiles(y_train)

y_train = prepare.update_price_quartiles(y_train, q25, q50, q75)
print("\ny_train value counts:")
print(y_train.value_counts())

y_eval = prepare.update_price_quartiles(y_eval, q25, q50, q75)
print("\ny_eval value counts:")
print(y_eval.value_counts())

X_train['similarAppsAvgPrice'] = prepare.update_price_quartiles(X_train['similarAppsAvgPrice'].copy(), q25, q50, q75)
print("\nX_train value counts:")
print(X_train['similarAppsAvgPrice'].value_counts())

X_eval['similarAppsAvgPrice'] = prepare.update_price_quartiles(X_eval['similarAppsAvgPrice'].copy(), q25, q50, q75)
print("\nX_eval value counts:")
print(X_eval['similarAppsAvgPrice'].value_counts())


pipe_steps = [
    ('baseline', BaselineClassifier(idx=list(X_train.columns).index('similarAppsAvgPrice')))
]
pipeline = Pipeline(pipe_steps)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_eval)

rs = pd.Series()
evaluate.onMetrics(rs, y_eval, y_pred)
print(rs)


y_train value counts:
2.0    4228
1.0    3951
3.0    3742
4.0    3177
Name: price, dtype: int64

y_eval value counts:
2.0    1395
3.0    1265
1.0    1261
4.0    1112
Name: price, dtype: int64

X_train value counts:
3.0    6808
4.0    4234
2.0    3238
1.0     818
Name: similarAppsAvgPrice, dtype: int64

X_eval value counts:
3.0    2188
4.0    1451
2.0    1114
1.0     280
Name: similarAppsAvgPrice, dtype: int64
score_precision_eval_mean    0.418996
score_precision_eval_std     0.068081
score_recall_eval_mean       0.397521
score_recall_eval_std        0.200513
score_f1_eval_mean           0.366470
score_f1_eval_std            0.126874
score_accuracy_eval          0.389032
dtype: float64
